In [1]:
import numpy as np
from Druckrohrleitung_class_file import Druckrohrleitung_class
import matplotlib.pyplot as plt

#importing pressure conversion function
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class

In [2]:
# define constants

    # for physics
g                   = 9.81                                          # [m/s²]    gravitational acceleration 
rho                 = 1000.                                         # [kg/m³]   density of water 
pUnit_calc          = 'Pa'                                          # [text]    DO NOT CHANGE! for pressure conversion in print statements and plot labels 
pUnit_conv          = 'mWS'                                         # [text]    for pressure conversion in print statements and plot labels


    # for Turbine
Tur_Q_nenn         = 0.85                                           # [m³/s]    nominal flux of turbine 
Tur_p_nenn         = pressure_conversion(10.6,'bar',pUnit_calc)     # [Pa]      nominal pressure of turbine 
Tur_closingTime    = 90.                                            # [s]       closing time of turbine


    # for PI controller
Con_targetLevel     = 8.                                            # [m]
Con_K_p             = 0.1                                           # [-]       proportional constant of PI controller
Con_T_i             = 10.                                         # [s]       timespan in which a steady state error is corrected by the intergal term
Con_deadbandRange   = 0.05                                          # [m]       Deadband range around targetLevel for which the controller does NOT intervene


    # for pipeline
Pip_length  = (535.+478.)                                           # [m]       length of pipeline
Pip_dia     = 0.9                                                   # [m]       diameter of pipeline
Pip_area    = Pip_dia**2/4*np.pi                                    # [m²]      crossectional area of pipeline
Pip_head    = 105.                                                  # [m]       hydraulic head of pipeline without reservoir
Pip_angle   = np.arcsin(Pip_head/Pip_length)                        # [rad]     elevation angle of pipeline 
Pip_n_seg   = 50                                                    # [-]       number of pipe segments in discretization
Pip_f_D     = 0.014                                                 # [-]       Darcy friction factor
Pip_pw_vel  = 500.                                                  # [m/s]     propagation velocity of the pressure wave (pw) in the given pipeline
    # derivatives of the pipeline constants
Pip_dx      = Pip_length/Pip_n_seg                                  # [m]       length of each pipe segment
Pip_dt      = Pip_dx/Pip_pw_vel                                     # [s]       timestep according to method of characteristics
Pip_nn      = Pip_n_seg+1                                           # [1]       number of nodes
Pip_x_vec   = np.arange(0,Pip_nn,1)*Pip_dx                         # [m]       vector holding the distance of each node from the upstream reservoir along the pipeline
Pip_h_vec   = np.arange(0,Pip_nn,1)*Pip_head/Pip_n_seg              # [m]       vector holding the vertival distance of each node from the upstream reservoir


    # for reservoir
Res_area_base       = 74.                                           # [m²]      total base are of the cuboid reservoir   
Res_area_out        = Pip_area                                      # [m²]      outflux area of the reservoir, given by pipeline area
Res_level_crit_lo   = 0.                                            # [m]       for yet-to-be-implemented warnings
Res_level_crit_hi   = np.inf                                        # [m]       for yet-to-be-implemented warnings
Res_dt_approx       = 1e-3                                          # [s]       approx. timestep of reservoir time evolution to ensure numerical stability (see Res_nt why approx.)
Res_nt              = max(1,int(Pip_dt//Res_dt_approx))             # [1]       number of timesteps of the reservoir time evolution within one timestep of the pipeline
Res_dt              = Pip_dt/Res_nt                                 # [s]       harmonised timestep of reservoir time evolution

    # for general simulation
flux_init       = Tur_Q_nenn/1.1                                    # [m³/s]    initial flux through whole system for steady state initialization  
level_init      = Con_targetLevel                                   # [m]       initial water level in upstream reservoir for steady state initialization
simTime_target  = 600.                                              # [s]       target for total simulation time (will vary slightly to fit with Pip_dt)
nt              = int(simTime_target//Pip_dt)                       # [1]       Number of timesteps of the whole system
t_vec           = np.arange(0,nt+1,1)*Pip_dt                          # [s]       time vector. At each step of t_vec the system parameters are stored


In [3]:
# create objects

# Upstream reservoir
reservoir = Ausgleichsbecken_class(Res_area_base,Res_area_out,Res_dt,Res_level_crit_lo,Res_level_crit_hi,rho)
reservoir.set_steady_state(flux_init,level_init)

# pipeline
pipe = Druckrohrleitung_class(Pip_length,Pip_dia,Pip_n_seg,Pip_angle,Pip_f_D,Pip_pw_vel,Pip_dt,pUnit_conv,rho)
pipe.set_steady_state(flux_init,level_init,Res_area_base,Pip_x_vec,Pip_h_vec)


In [ ]:
reservoir.get_info(full=True)
pipe.get_info(full=True)

In [4]:
# initialization for timeloop

level_vec       = np.zeros_like(t_vec)
level_vec[0]    = reservoir.get_current_level()

# prepare the vectors in which the pressure and velocity distribution in the pipeline from the previous timestep are stored
v_old = pipe.get_current_velocity_distribution()
p_old = pipe.get_current_pressure_distribution()

# prepare the vectors in which the temporal evolution of the boundary conditions are stored
    # keep in mind, that the velocity at the turbine and the pressure at the reservoir are set manually and
        # through the time evolution of the reservoir respectively  
    # the pressure at the turbine and the velocity at the reservoir are calculated from the method of characteristics
v_boundary_res  = np.zeros_like(t_vec)
v_boundary_tur  = np.zeros_like(t_vec)
p_boundary_res  = np.zeros_like(t_vec)
p_boundary_tur  = np.zeros_like(t_vec)

# set the boundary conditions for the first timestep
v_boundary_res[0]   = v_old[0]
v_boundary_tur[0]   = v_old[-1] 
p_boundary_res[0]   = p_old[0]
p_boundary_tur[0]   = p_old[-1]


In [5]:
%matplotlib qt5
fig1,axs1 = plt.subplots(2,1)
axs1[0].set_title('Pressure distribution in pipeline')
axs1[0].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[0].set_ylabel(r'$p$ [mWS]')
axs1[0].set_ylim([0.9*np.min(pressure_conversion(p_old,'Pa',pUnit_conv)),1.1*np.max(pressure_conversion(p_old,'Pa',pUnit_conv))])
lo_00, = axs1[0].plot(Pip_x_vec,pressure_conversion(p_old,'Pa',pUnit_conv),marker='.')

axs1[1].set_title('Velocity distribution in pipeline')
axs1[1].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[1].set_ylabel(r'$v$ [m/s]')
lo_01, = axs1[1].plot(Pip_x_vec,v_old,marker='.')
axs1[1].autoscale()
# axs1[1].set_ylim([0.9*np.min(v_old),1.1*np.max(v_boundary_res)])

fig1.tight_layout()
plt.pause(1)

In [6]:
for it_pipe in range(1,nt+1):
# for each pipeline timestep, execute nt_eRK4 timesteps of the reservoir code
    # set initial conditions for the reservoir time evolution calculted with e-RK4
    reservoir.set_pressure(p_old[0],display_warning=False)
    reservoir.set_outflux(v_old[0]*Pip_area,display_warning=False)
    # calculate the time evolution of the reservoir level within each pipeline timestep to avoid runaway numerical error
    for it_res in range(Res_nt):
        reservoir.timestep_reservoir_evolution()                                                             
    level_vec[it_pipe] = reservoir.get_current_level() 

    
    # set boundary conditions for the next timestep of the characteristic method
    p_boundary_res[it_pipe] = reservoir.get_current_pressure()
    v_boundary_tur[it_pipe] = flux_init/Pip_area

    # the the boundary conditions in the pipe.object and thereby calculate boundary pressure at turbine
    pipe.set_boundary_conditions_next_timestep(p_boundary_res[it_pipe],v_boundary_tur[it_pipe])
    p_boundary_tur[it_pipe] = pipe.get_current_pressure_distribution()[-1]
    v_boundary_res[it_pipe] = pipe.get_current_velocity_distribution()[0]

    # perform the next timestep via the characteristic method
    pipe.timestep_characteristic_method()

    # prepare for next loop
    p_old = pipe.get_current_pressure_distribution()
    v_old = pipe.get_current_velocity_distribution()

    # plot some stuff
        # remove line-objects to autoscale axes (there is definetly a better way, but this works ¯\_(ツ)_/¯ )
    lo_00.remove()
    lo_01.remove()
    # lo_02.remove()
        # plot new pressure and velocity distribution in the pipeline
    lo_00, = axs1[0].plot(Pip_x_vec,pressure_conversion(p_old,'Pa', pUnit_conv),marker='.',c='blue')
    lo_01, = axs1[1].plot(Pip_x_vec,v_old,marker='.',c='blue')
    
    fig1.suptitle(str(round(t_vec[it_pipe],2)) + '/' + str(round(t_vec[-1],2)))
    fig1.canvas.draw()
    fig1.tight_layout()
    plt.pause(0.000001)

reservoir.get_info(full=True)
pipe.get_info(full=True)

KeyboardInterrupt: 

In [12]:
fig2,axs2 = plt.subplots(2,2)
axs2[0,0].set_title('Pressure Reservoir')
axs2[0,0].plot(t_vec,pressure_conversion(p_boundary_res,pUnit_calc,pUnit_conv))
axs2[0,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[0,0].set_ylabel(r'$p$ [mWS]')
axs2[0,0].set_ylim([0.9*np.min(pressure_conversion(p_boundary_res,pUnit_calc,pUnit_conv)),1.1*np.max(pressure_conversion(p_boundary_res,pUnit_calc,pUnit_conv))])

axs2[0,1].set_title('Velocity Reservoir')
axs2[0,1].plot(t_vec,v_boundary_res)
axs2[0,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[0,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs2[0,1].set_ylim([0.9*np.min(v_boundary_res),1.1*np.max(v_boundary_res)])

axs2[1,0].set_title('Pressure Turbine')
axs2[1,0].plot(t_vec,pressure_conversion(p_boundary_tur,pUnit_calc,pUnit_conv))
axs2[1,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[1,0].set_ylabel(r'$p$ [mWS]')
axs2[1,0].set_ylim([0.9*np.min(pressure_conversion(p_boundary_tur,pUnit_calc,pUnit_conv)),1.1*np.max(pressure_conversion(p_boundary_tur,pUnit_calc,pUnit_conv))])

axs2[1,1].set_title('Velocity Turbine')
axs2[1,1].plot(t_vec,v_boundary_tur)
axs2[1,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs2[1,1].set_ylabel(r'$v$ [$\mathrm{m}/\mathrm{s}$]')
axs2[1,1].set_ylim([0.95*np.min(v_boundary_tur),1.05*np.max(v_boundary_tur)])

fig2.tight_layout()
plt.show()